In [52]:
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace  
from langchain_core.tools import tool, InjectedToolArg
from dotenv import load_dotenv
from typing import Annotated
from langchain_core.messages import HumanMessage
import requests

In [53]:
load_dotenv()

True

In [54]:
llm = HuggingFaceEndpoint(
    repo_id="openai/gpt-oss-20b",
    task ="text-generation",
    # temperature= 2
)
model = ChatHuggingFace(llm = llm)

In [ ]:
# getting conversion rete

@tool
def get_conversion_rate(base_currency: str, target_currency: str) -> float:
    """This function fetches the currency conversion factor between a given base currency and a target currency """ 
    url = f"https://v6.exchangerate-api.com/v6/82f231ae2c5c8b901778eb76/pair/{base_currency}/{target_currency}" 
    response = requests.get(url=url)
    return response.json()

In [56]:
result = get_conversion_rate.invoke({'base_currency':'USD', 'target_currency':'NPR'})
print(result)

{'result': 'success', 'documentation': 'https://www.exchangerate-api.com/docs', 'terms_of_use': 'https://www.exchangerate-api.com/terms', 'time_last_update_unix': 1769212802, 'time_last_update_utc': 'Sat, 24 Jan 2026 00:00:02 +0000', 'time_next_update_unix': 1769299202, 'time_next_update_utc': 'Sun, 25 Jan 2026 00:00:02 +0000', 'base_code': 'USD', 'target_code': 'NPR', 'conversion_rate': 146.8897}


In [57]:
# converting tool
# making that InjectedToolArg so that ai don't place there random value
# 
@tool
def convert_value(base_currency_amount: int, conversion_rate: Annotated[float,InjectedToolArg]) -> float:
    """
  given a currency conversion rate this function calculates the target currency value from a given base currency amount
  """
    return base_currency_amount*conversion_rate

In [58]:
convert_value.args

{'base_currency_amount': {'title': 'Base Currency Amount', 'type': 'integer'}}

In [59]:
model_with_tool = model.bind_tools([get_conversion_rate, convert_value])

In [60]:
messages = [HumanMessage('What is the conversion factor between nepal currency and USD, and based on that can you convert 10 usd to nepal currency ')]

In [61]:
ai_message = model_with_tool.invoke(messages)

In [62]:
ai_message

AIMessage(content='', additional_kwargs={'tool_calls': [{'function': {'arguments': '{"base_currency":"USD","target_currency":"NPR"}', 'name': 'get_conversion_rate', 'description': None}, 'id': 'fc_f995d2f9-725a-4e3b-9628-cb5f6c5ce5be', 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 220, 'prompt_tokens': 201, 'total_tokens': 421}, 'model_name': 'openai/gpt-oss-20b', 'system_fingerprint': 'fp_92d51d08e5', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019bee71-86ed-7d82-91b7-ec2b96aaf00b-0', tool_calls=[{'name': 'get_conversion_rate', 'args': {'base_currency': 'USD', 'target_currency': 'NPR'}, 'id': 'fc_f995d2f9-725a-4e3b-9628-cb5f6c5ce5be', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 201, 'output_tokens': 220, 'total_tokens': 421})

In [63]:
ai_message.tool_calls

[{'name': 'get_conversion_rate',
  'args': {'base_currency': 'USD', 'target_currency': 'NPR'},
  'id': 'fc_f995d2f9-725a-4e3b-9628-cb5f6c5ce5be',
  'type': 'tool_call'}]

In [64]:
messages.append(ai_message)

In [65]:
import json

for tool_call in ai_message.tool_calls:
    if tool_call['name'] == 'get_conversion_rate':
        tool_message1 = get_conversion_rate.invoke(tool_call)
        conversion_rate = json.loads(tool_message1.content)['conversion_rate']
        messages.append(tool_message1)
        
    if tool_call['name'] == 'convert_value':
        tool_call['args']['conversion_rate'] = conversion_rate
        tool_message2 = convert_value.invoke(tool_call)
        messages.append(tool_message2)
    

In [66]:
messages

[HumanMessage(content='What is the conversion factor between nepal currency and USD, and based on that can you convert 10 usd to nepal currency ', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'function': {'arguments': '{"base_currency":"USD","target_currency":"NPR"}', 'name': 'get_conversion_rate', 'description': None}, 'id': 'fc_f995d2f9-725a-4e3b-9628-cb5f6c5ce5be', 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 220, 'prompt_tokens': 201, 'total_tokens': 421}, 'model_name': 'openai/gpt-oss-20b', 'system_fingerprint': 'fp_92d51d08e5', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019bee71-86ed-7d82-91b7-ec2b96aaf00b-0', tool_calls=[{'name': 'get_conversion_rate', 'args': {'base_currency': 'USD', 'target_currency': 'NPR'}, 'id': 'fc_f995d2f9-725a-4e3b-9628-cb5f6c5ce5be', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 201, 'output_tokens': 220, 'total_

In [67]:
model_with_tool.invoke(messages).content

'**Conversion factor**  \n1\u202fUSD\u202f=\u202f146.8897\u202fNPR\n\n**10\u202fUSD to Nepali rupees**  \n\n10\u202fUSD\u202f×\u202f146.8897\u202fNPR/USD\u202f≈\u202f**1\u202f468.90\u202fNPR**\n\n(rounded to two decimal places)'